In [1]:
import sys

sys.path.append("..")

In [2]:
import numpy as np 
x = np.array([1, 2, 3])
np.save('test.npy', x)

x = np.load('test.npy')
print(x)

[1 2 3]


In [3]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])
np.savez('test.npz', x1=x1, x2=x2)

arrays = np.load('test.npz')

x1 = arrays['x1']
x2 = arrays['x2']

print(x1)
print(x2)

[1 2 3]
[4 5 6]


In [4]:
x1 = np.array([1, 2, 3])
x2 = np.array([4, 5, 6])

data =  {
    'x1': x1, 'x2': x2
}
np.savez('test.npz', **data)

arrays = np.load('test.npz')
x1 = arrays['x1']
x2 = arrays['x2']

print(x1)
print(x2)

[1 2 3]
[4 5 6]


In [5]:
from mytorch.layers import Layer
from mytorch.core import Parameter

layer = Layer()

l1 = Layer()
l1.p1 = Parameter(np.array(1))

layer.l1 = l1
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))

params_dict = {}
layer._flatten_params(params_dict)

print(params_dict)

{'p2': variable(2), 'p3': variable(3), 'l1/name': variable(1)}


In [6]:
import time

import mytorch
import mytorch.functions as F
from mytorch import optimizers, DataLoader
from mytorch.models import MLP

max_epoch = 5
batch_size = 100

hidden_size = 1000
train_set = mytorch.datasets.MNIST(train=True)
test_set = mytorch.datasets.MNIST(train=False)

train_loader = DataLoader(train_set, batch_size)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

model = MLP((hidden_size, 10))
optim = optimizers.SGD(model.parameters)

if mytorch.cuda.gpu_enable:
    print("using GPU")
    train_loader.to_gpu()
    test_loader.to_gpu()
    model.to_gpu()


for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    for x, t in train_loader:
        pred = model(x)
        loss = F.softmax_cross_entropy(pred, t)
        acc = F.accuracy(pred, t)
        
        model.zerograd()
        loss.backward()
        optim.step()
        
        sum_loss += float(loss.data) * len(t)
        sum_acc += float(acc.data) * len(t)
    print(f"[{epoch+1}/{max_epoch}] train_loss: {sum_loss / len(train_set)}, "
          f"accuracy: {sum_acc / len(train_set)}")
    
    sum_loss, sum_acc = 0, 0
    with mytorch.no_grad():
        for x, t in test_loader:
            pred = model(x)
            loss = mytorch.functions.softmax_cross_entropy(pred, t)
            acc = mytorch.functions.accuracy(pred, t)
            sum_loss += float(loss.data) * len(t)
            sum_acc += float(acc.data) * len(t)
        print(f"test_loss: {sum_loss / len(test_set)}, "
              f"accuracy: {sum_acc / len(test_set)}")
    print(f"elapse times : {time.time() - start}")

model.save_weights('my_mlp.npz')

using GPU
[1/5] train_loss: 1.922821028585693, accuracy: 0.5528666666666666
test_loss: 1.548512612586489, accuracy: 0.716
elapse times : 3.554969310760498
[2/5] train_loss: 1.2854120410220773, accuracy: 0.7737333333333334
test_loss: 1.0428398126816574, accuracy: 0.8202
elapse times : 3.2473764419555664
[3/5] train_loss: 0.9235410265863565, accuracy: 0.819
test_loss: 0.7904263709944886, accuracy: 0.8403
elapse times : 3.249432325363159
[4/5] train_loss: 0.7388662404325429, accuracy: 0.8405666666666667
test_loss: 0.6565948274777448, accuracy: 0.8541
elapse times : 3.2711293697357178
[5/5] train_loss: 0.6345962548385558, accuracy: 0.8534166666666667
test_loss: 0.5755189209622363, accuracy: 0.8688
elapse times : 3.2383289337158203


In [8]:
new_model = MLP((hidden_size, 10))
new_model.load_weights('my_mlp.npz')


if mytorch.cuda.gpu_enable:
    print("using GPU")
    new_model.to_gpu()

sum_loss, sum_acc = 0, 0
with mytorch.no_grad():
    for x, t in test_loader:
        pred = new_model(x)
        loss = mytorch.functions.softmax_cross_entropy(pred, t)
        acc = mytorch.functions.accuracy(pred, t)
        sum_loss += float(loss.data) * len(t)
        sum_acc += float(acc.data) * len(t)
    print(f"test_loss: {sum_loss / len(test_set)}, "
          f"accuracy: {sum_acc / len(test_set)}")

using GPU
test_loss: 0.5698821171979376, accuracy: 0.8607
